<a href="https://colab.research.google.com/github/emschenn/mlg_hw1/blob/master/MLG_hw1_DrBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0.html
  Using cached https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0/torch_scatter-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-scatter 2.0.4
    Uninstalling torch-scatter-2.0.4:
      Successfully uninstalled torch-scatter-2.0.4
Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0.html
  Using cached https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0/torch_sparse-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-sparse 0.6.1
    Uninstalling torch-sparse-0.6.1:
      Successfully uninstalled torch-sparse-0.6.1
Looking in links: https://pytorch-geometric.com/whl/torch-1.4.0.html
  Using cached https://s3.eu-central-1.amazonaws.com/pytorch-geometric.com/whl/torch-1.4.0/torch_cluster-latest%2Bcu101-cp36-cp36m-linux_x86_64.whl
  Found existing installation: torch-cluster 1.5.3
    Uninstalling torch

In [23]:
# prepare model
import torch
import torch.nn as nn
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
dim =128
class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')  # "Add" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)
        # Step 3: Compute normalization
        row, col = edge_index
        deg = degree(row, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        # Step 4-6: Start propagating messages.
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x, norm=norm)

    def message(self, x_j, norm):
        # x_j has shape [E, out_channels]
        # Step 4: Normalize node features.
        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels]
        # Step 6: Return new node embeddings.
        return aggr_out

class DrBC(nn.Module):
    def __init__(self,):
        super(DrBC, self).__init__()
        # Encoder
        self.fc1 = nn.Linear(3, dim)
        self.relu = nn.LeakyReLU()

        self.gcn1 = GCNConv(dim, dim)
        self.gru1 = nn.GRU(dim, dim)

        self.gcn2 = GCNConv(dim, dim)
        self.gru2 = nn.GRU(dim, dim)

        self.gcn3 = GCNConv(dim, dim)
        self.gru3 = nn.GRU(dim, dim)

        self.gcn4 = GCNConv(dim, dim)
        self.gru4 = nn.GRU(dim, dim)

        self.gcn5 = GCNConv(dim, dim)
        self.gru5 = nn.GRU(dim, dim)

        # Decoder
        self.fc2 = nn.Linear(dim, int(dim/2))
        self.relu2 = nn.LeakyReLU()
        self.fc3 = nn.Linear(int(dim/2), 1)

    def forward(self, x, edge_index):
        x = self.fc1(x)
        x = self.relu(x)        
        x_n = self.gcn1(x, edge_index)
        x1, _ = self.gru1(x_n.view(1, *x_n.shape), x.view(1, *x.shape))
        x_n = self.gcn2(x1[0], edge_index)
        x2, _ = self.gru2(x_n.view(1, *x_n.shape), x1)
        x_n = self.gcn3(x2[0], edge_index)
        x3, _ = self.gru3(x_n.view(1, *x_n.shape), x2)
        x_n = self.gcn4(x3[0], edge_index)
        x4, _ = self.gru4(x_n.view(1, *x_n.shape), x3)
        x_n = self.gcn5(x4[0], edge_index)
        x5, _ = self.gru5(x_n.view(1, *x_n.shape), x4)
        
        # max
        l = [x1[0],x2[0],x3[0],x4[0],x5[0]]
        l = torch.stack(l)
        x = torch.max(l, dim=0).values
        # l = torch.tensor(l) 

        # decoder
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

"""
model = DrBC()
x = torch.tensor([[2, 1, 1],[2, 1, 1],[2, 1, 1],[2, 1, 1]],dtype=torch.float)
edge_index = torch.tensor([[0,1,2],[1,2,3]],dtype=torch.long)
print(model(x, edge_index))
"""

'\nmodel = DrBC()\nx = torch.tensor([[2, 1, 1],[2, 1, 1],[2, 1, 1],[2, 1, 1]],dtype=torch.float)\nedge_index = torch.tensor([[0,1,2],[1,2,3]],dtype=torch.long)\nprint(model(x, edge_index))\n'

In [34]:
# prepare data
import numpy as np
import networkx as nx
import random 
import math

class Graph():
  def __init__(self,batch_size):
    self.graph_list=[]
    for x in range(batch_size):
      g = nx.powerlaw_cluster_graph(n=random.randint(150,200) , m=4, p=0.05, seed=None)
      self.graph_list.append(g)

  def get_deg_list(self):
    deg_list = []
    for g in self.graph_list:
      for x in range(g.number_of_nodes()):
        deg_list.append([g.degree[x],1,1])
    return torch.Tensor(deg_list).cuda()

  def get_edge_index(self):
    s_list,t_list,en = [],[],0
    for g in self.graph_list:
      for e in g.edges():
        s,t = e
        s_list.append(s+en)
        t_list.append(t+en)
      en += g.number_of_nodes()
    edge_index=[s_list+t_list,t_list+s_list]
    return torch.tensor(edge_index,dtype=torch.long).cuda()

  def get_bc_value(self):
    bc_value = []
    for g in self.graph_list:
      bc_value += list(nx.betweenness_centrality(g).values())
    for i, x in enumerate(bc_value):
      bc_value[i] = math.log(x+1e-8)
    return torch.Tensor(bc_value).cuda()

  def get_pair_index(self):
    n_list1,n_list2,pair_index,en = [],[],[],0
    for g in self.graph_list:
      for x in range(g.number_of_nodes()):
        n_list1 += [en+x,en+x,en+x,en+x,en+x]
        n_list2 += [en+x,en+x,en+x,en+x,en+x]
      random.shuffle(n_list1)
      random.shuffle(n_list2)
      for i,j in zip(n_list1,n_list2):
        pair_index.append([i,j])
      n_list1,n_list2=[],[]
    return torch.tensor(pair_index, dtype=torch.long).cuda()

"""
g = Graph(16)
print(g.get_deg_list().shape)
print(g.get_edge_index().shape)
print(g.get_bc_value().shape)
print(g.get_pair_index())
"""

'\ng = Graph(16)\nprint(g.get_deg_list().shape)\nprint(g.get_edge_index().shape)\nprint(g.get_bc_value().shape)\nprint(g.get_pair_index())\n'

In [38]:
# start training
from torch.optim import Adam
import torch.nn.functional as F
iteration = 10000

def train():
  model = DrBC()
  model = model.cuda()
  optimizer = Adam(params=model.parameters(), lr=0.0001)
  g = Graph(16)
  for iter in range(iteration):
    if iter % 500 == 0:
      g = Graph(16)
      bc = g.get_bc_value()
    outs = model(g.get_deg_list(),g.get_edge_index())
    pair = g.get_pair_index()
    pred = outs[pair[:, 0]] - outs[pair[:, 1]]
    gt = torch.sigmoid((bc[pair[:, 0]] - bc[pair[:, 1]]))
    gt = gt.view(-1, 1)
    loss = F.binary_cross_entropy_with_logits(pred, gt, reduction="sum")
    if iter % 500 == 0:
      print("[{}/{}] Loss:{:}".format(iter, iteration, loss.item()))
    loss.backward()
    optimizer.step()
  torch.save(model.state_dict(), "./weight.pth")
  return model

model = train()

[0/10000] Loss:11510.470703125
[500/10000] Loss:7094.15380859375
[1000/10000] Loss:6913.81103515625
[1500/10000] Loss:8417.693359375
[2000/10000] Loss:7241.21875
[2500/10000] Loss:7136.6103515625
[3000/10000] Loss:6954.431640625
[3500/10000] Loss:6990.8388671875
[4000/10000] Loss:6958.705078125
[4500/10000] Loss:7018.30078125
[5000/10000] Loss:7010.66552734375
[5500/10000] Loss:7357.2265625
[6000/10000] Loss:6699.919921875
[6500/10000] Loss:7463.880859375
[7000/10000] Loss:6715.58544921875
[7500/10000] Loss:6641.1826171875
[8000/10000] Loss:7034.2861328125
[8500/10000] Loss:6935.42529296875
[9000/10000] Loss:7610.1943359375
[9500/10000] Loss:7369.15283203125


In [0]:
# Read file
import urllib.request  

class readFile():
  def __init__(self,file):
    if file == 'y':
      url1 = 'https://raw.githubusercontent.com/emschenn/mlg_hw1/master/hw1_data/youtube/com-youtube.txt' 
      url2 = 'https://raw.githubusercontent.com/emschenn/mlg_hw1/master/hw1_data/youtube/com-youtube_score.txt' 
    else:
      url1 = 'https://raw.githubusercontent.com/emschenn/mlg_hw1/master/hw1_data/Synthetic/5000/' + file + '.txt'
      url2 = 'https://raw.githubusercontent.com/emschenn/mlg_hw1/master/hw1_data/Synthetic/5000/' + file + '_score.txt'
    self.bc_value,s_list,t_list,self.deg_list,n = [],[],[],[],0
    for line in urllib.request.urlopen(url2):
      _,v = line.decode('utf-8').split()
      self.bc_value.append([n,math.log(float(v)+1e-8)])
      n += 1
    for x in range(len(self.bc_value)):
      self.deg_list.append([0,1,1])
    for line in urllib.request.urlopen(url1):
      s,t = line.decode('utf-8').split()
      s,t = int(s),int(t)
      s_list.append(s)
      t_list.append(t)
      self.deg_list[s][0]+=1
      self.deg_list[t][0]+=1
    self.edge_index=[s_list+t_list,t_list+s_list]

  def get_deg_list(self):
    # print(self.deg_list)
    return torch.Tensor(self.deg_list).cuda()

  def get_edge_index(self):
    # print(self.edge_index)
    return torch.tensor(self.edge_index,dtype=torch.long).cuda()

  def get_bc_value(self):
    # print(self.bc_value)
    return self.bc_value


In [40]:
# Evaluation
def takeSecond(elem):
    return elem[1]

def topN_accuracy(file,outs,n):
  predict_value,bc_value = [],[]
  for i,j in enumerate(outs.tolist()):
    predict_value.append([i,*j])
  bc_value = f.get_bc_value()
  bc_value.sort(key = takeSecond,reverse = True)
  predict_value.sort(key = takeSecond,reverse = True)
  p,t = [],[]
  for x in range(int(len(predict_value)*n/100)):
    p.append(predict_value[x][0])
    t.append(bc_value[x][0])
  return(len(set(t)&set(p)) / len(p))

import scipy.stats as stats
def kendall_tau(file,outs):
  predict_value,bc_value = [],[]
  for i,j in enumerate(outs.tolist()):
    predict_value.append(*j)
  for i in file.get_bc_value():
    bc_value.append(i[1])
  # print(predict_value)
  # print(bc_value)
  tau, _ = stats.kendalltau(predict_value, bc_value)
  return(tau)

read_data = '1'

if(read_data == 'youtube'):
  f = readFile('y')
  model = model.cpu()
  t = f.get_deg_list().cpu()
  t1 = f.get_edge_index().cpu()
  with torch.no_grad():
    outs = model(t,t1)
else:
  f = readFile(read_data)
  outs = model(f.get_deg_list(),f.get_edge_index())

print(read_data)
  # Top-N % accuracy
print("top-1% accuracy: {}".format(topN_accuracy(f,outs,n=1)))
print("top-5% accuracy: {}".format(topN_accuracy(f,outs,n=5)))
print("top-10% accuracy: {}".format(topN_accuracy(f,outs,n=10)))

  # Kendall tau
print("Kendall tau: {}".format(kendall_tau(f,outs)))


1
top-1% accuracy: 0.94
top-5% accuracy: 0.924
top-10% accuracy: 0.856
Kendall tau: 0.6282254079832005
